## 决策变量与约束

In [1]:
from gurobipy import *

# 有十个工作日需要安排工作
Shifts = ["day1", "day2", "day3", "day4", "day5", "day6", "day7", "day8", "day9", "day10"]

# 每天需要的员工数量
Shifts_req = [2, 3, 4, 5, 4, 3, 5, 2, 5, 3]

# 一共有六位员工
Employees = ['Ken', 'Emma', 'Oli', 'Ava', 'Bob', 'Jms']

# 员工可用性矩阵，Availability[i][j]表示第i位员工在第j天有空
Availability = [[1, 1, 0, 1, 1, 0, 1, 1, 0, 1],
                [0, 0, 1, 1, 1, 0, 1, 1, 1, 1],
                [0, 1, 1, 0, 1, 1, 0, 1, 1, 1],
                [1, 0, 1, 1, 1, 0, 1, 1, 1, 0],
                [1, 1, 1, 0, 0, 1, 0, 1, 1, 0],
                [0, 1, 1, 1, 0, 1, 1, 0, 1, 1],]

# 建立模型
model = Model('Work_Assignment')

# 决策变量1：员工i是否要在第j天工作
x = {}
for i in range(len(Employees)):
    for j in range(len(Shifts)):
        x[i,j] = model.addVar(lb=0, ub=Availability[i][j], vtype=GRB.BINARY, name='x_'+str(i)+'_'+str(j))

# 决策变量2：每个Shift添加一个松弛变量，用于确保当员工不足时，每个Shift都能满足
slacks = {}
for s in range(len(Shifts)):
    slacks[s] = model.addVar(name='Slack')

# 决策变量3：总松弛数量
total_slack = model.addVar(name='totSlack')

# 决策变量3：每个员工的工作天数
total_shifts = {}
for e in range(len(Employees)):
    total_shifts[e] = model.addVar(name='totShifts_'+str(e))


# 决策变量4与5：用于获得员工最大工作天数与最少工作天数
minShift = model.addVar(name='minShift')
maxShift = model.addVar(name='maxShift')

# 约束1：每个Shift都要被满足
for i, s in enumerate(Shifts_req):
    expr = LinExpr()
    for e in range(len(Employees)):
        expr.addTerms(1, x[e, i])
    model.addConstr(expr + slacks[i] == s)

# 约束2：获得总松弛数量
expr = LinExpr()
for s in range(len(slacks)):
    expr.addTerms(1, slacks[s])
model.addConstr(total_slack == expr, name='total_ Slack')

# 约束3：计算每个员工的工作天数
for e in range(len(Employees)):
    expr = LinExpr()
    for s in range(len(Shifts)):
        expr.addTerms(1, x[e, s])
    model.addConstr(total_shifts[e] == expr, name='total_Shifts_'+str(e))


total_shifts_list = []
for e in range(len(Employees)):
    total_shifts_list.append(total_shifts[e])

# 约束4：所有员工中最少的工作天数
model.addGenConstrMin(minShift, total_shifts_list, name='minShift')

# 约束5：所有员工中最多的工作天数
model.addGenConstrMax(maxShift, total_shifts_list, name='maxShift')

<gurobi.GenConstr *Awaiting Model Update*>

## 分级法

In [2]:
# 设置所有目标函数的 global sense
model.ModelSense = GRB.MINIMIZE

# 首要目标，最小化员工缺失
model.setObjectiveN(total_slack, index=0, priority=2, name='TotalSlack')

# 次要目标函数，最小化员工间的工作时长差异
model.setObjectiveN(maxShift - minShift, index=1, priority=1, name='Fairness')

model.Params.ObjNumber = 0
model.ObjNCon = 1000

model.write('work_assignment.mps')
model.optimize()

print('Total slack required: ' + str(total_slack.X))
for i, e in enumerate(Employees):
    print(e + ' worked ' + str(total_shifts[i].X) + ' shifts')

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 17 rows, 79 columns and 147 nonzeros
Model fingerprint: 0x1e0211b5
Model has 2 general constraints
Variable types: 19 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 5e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve added 8 rows and 0 columns
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 25 rows and 78 columns
-------------------------------------------------------------

## 混合法

In [3]:
# 设置所有目标函数的 global sense
model.ModelSense = GRB.MINIMIZE

# 首要目标函数
model.setObjectiveN(total_slack, index=0, weight=1, name='TotalSlack')

# 次要目标函数
model.setObjectiveN(maxShift - minShift, index=1, weight=1, name='Fairness')

model.write('work_assignment.lp')
model.optimize()

print('总员工缺失数为: ' + str(total_slack.X))

for i, e in enumerate(Employees):
    print(e + ' 工作 ' + str(total_shifts[i].X) + ' 天')

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 17 rows, 79 columns and 147 nonzeros
Model fingerprint: 0x0128d399
Model has 2 general constraints
Variable types: 19 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 5e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 17 rows, 79 columns and 147 nonzeros
Model fingerprint: 0x99be6da7
Model h

In [4]:
model.optimize

# 获得决策变量集合
x = model.getVars()

# 确保模型已经获得最优解
assert model. Status == GRB.Status.OPTIMAL

# 获得模型解的数量与目标函数的数量
sol_num = model.SolCount
obj_num = model.NumObj
print('该模型有', sol_num, '个解。')
print('该模型有', obj_num, '个目标函数。\n')

# 打印第s个可行解对应的第o个目标函数值
solution = []
for s in range(sol_num):
    # 设置将要查询的可行集的索引值
    model.Params.SolutionNumber = s

    for o in range(obj_num):
        # 设置将要查询的目标函数值的索引值
        model.Params.ObjNumber = o
        print('第', o,'个目标函数值为：', model.ObjNVal, end='\n')
    print('')

该模型有 2 个解。
该模型有 2 个目标函数。

第 0 个目标函数值为： 2.0
第 1 个目标函数值为： 1.0

第 0 个目标函数值为： 15.0
第 1 个目标函数值为： 2.0



In [5]:
from gurobipy import *

# 有十个工作日需要安排工作
Shifts = ["day1", "day2", "day3", "day4", "day5", "day6", "day7", "day8", "day9", "day10"]

# 每天需要的员工数量
Shifts_req = [2, 3, 4, 5, 4, 3, 5, 2, 5, 3]

# 一共有六位员工
Employees = ['Ken', 'Emma', 'Oli', 'Ava', 'Bob', 'Jms']

# 员工可用性矩阵，Availability[i][j]表示第i位员工在第j天有空
Availability = [[1, 1, 0, 1, 1, 0, 1, 1, 0, 1],
                [0, 0, 1, 1, 1, 0, 1, 1, 1, 1],
                [0, 1, 1, 0, 1, 1, 0, 1, 1, 1],
                [1, 0, 1, 1, 1, 0, 1, 1, 1, 0],
                [1, 1, 1, 0, 0, 1, 0, 1, 1, 0],
                [0, 1, 1, 1, 0, 1, 1, 0, 1, 1],]

# 建立模型
model = Model('Work_Assignment')

# 决策变量1：员工i是否要在第j天工作
x = {}
for i in range(len(Employees)):
    for j in range(len(Shifts)):
        x[i,j] = model.addVar(lb=0, ub=Availability[i][j], vtype=GRB.BINARY, name='x_'+str(i)+'_'+str(j))

# 决策变量2：每个Shift添加一个松弛变量，用于确保当员工不足时，每个Shift都能满足
slacks = {}
for s in range(len(Shifts)):
    slacks[s] = model.addVar(name='Slack')

# 决策变量3：总松弛数量
total_slack = model.addVar(name='totSlack')

# 决策变量3：每个员工的工作天数
total_shifts = {}
for e in range(len(Employees)):
    total_shifts[e] = model.addVar(name='totShifts_'+str(e))


# 决策变量4与5：用于获得员工最大工作天数与最少工作天数
minShift = model.addVar(name='minShift')
maxShift = model.addVar(name='maxShift')

# 约束1：每个Shift都要被满足
for i, s in enumerate(Shifts_req):
    expr = LinExpr()
    for e in range(len(Employees)):
        expr.addTerms(1, x[e, i])
    model.addConstr(expr + slacks[i] == s)

# 约束2：获得总松弛数量
expr = LinExpr()
for s in range(len(slacks)):
    expr.addTerms(1, slacks[s])
model.addConstr(total_slack == expr, name='total_ Slack')

# 约束3：计算每个员工的工作天数
for e in range(len(Employees)):
    expr = LinExpr()
    for s in range(len(Shifts)):
        expr.addTerms(1, x[e, s])
    model.addConstr(total_shifts[e] == expr, name='total_Shifts_'+str(e))


total_shifts_list = []
for e in range(len(Employees)):
    total_shifts_list.append(total_shifts[e])

# 约束4：所有员工中最少的工作天数
model.addGenConstrMin(minShift, total_shifts_list, name='minShift')

# 约束5：所有员工中最多的工作天数
model.addGenConstrMax(maxShift, total_shifts_list, name='maxShift')

# 设置所有目标函数的 global sense
model.ModelSense = GRB.MINIMIZE

# 首要目标函数
model.setObjectiveN(total_slack, index=0, weight=1, name='TotalSlack')

# 次要目标函数
model.setObjectiveN(maxShift - minShift, index=1, weight=0.02, name='Fairness')

model.write('work_assignment.lp')
model.optimize()

print('Total slack required: ' + str(total_slack.X))

for i, e in enumerate(Employees):
    print(e + ' worked ' + str(total_shifts[i].X) + ' shifts')

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 17 rows, 79 columns and 147 nonzeros
Model fingerprint: 0x34242478
Model has 2 general constraints
Variable types: 19 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 5e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 17 rows, 79 columns and 147 nonzeros
Model fingerprint: 0x46ab08e7
Model h